In [72]:
import numpy as np
import pandas as pd

import xlrd
import time
import xlsxwriter as xlsxwriter
import csv
import math
from datetime import datetime
import re

## 0. 파일 불러오기

+ 크롤링을 완료한 데이터
+ 영화명, 개봉일, 매출액, 누적매출액, 관객수, 누적관객수, 스크린수, 상영횟수, 대표국적, 배급사, 등급, 장르, 감독, 배우

In [26]:
# 출력 데이터수 설정

pd.set_option('display.max_rows',1000)

In [27]:
# 파일 불러오기

df_raw = pd.read_excel("crawl_total.xlsx", index_col = False, thousands= ',')
df = df_raw
df = df.loc[:, "영화명": "배우"]
df

,영화명,개봉일,매출액,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,배급사,등급,장르,감독,배우
0,태극기 휘날리며,2004-02-05,15687180500,15687180500,2544911,2544911,115,22375,한국,"(주)다자인소프트,(주)쇼박스",15세관람가,"전쟁,드라마",강제규,"장동건,원빈,이은주,공형진,조원희,이장욱,이영란,정희준,박유신,김효열,김효준,최재혁..."
1,트로이,2004-05-21,12777346000,12777346000,2001293,2001293,177,21245,미국,워너브러더스 코리아(주),15세관람가,사극,볼프강 페터젠,"다이앤 크루거,올랜도 블룸,에릭 바나"
2,내 머리 속의 지우개,2004-11-05,11919939500,11919939500,1885827,1885827,206,29532,한국,CJ ENM,12세관람가,멜로/로맨스,이재한,"정우성,손예진,데이비드 맥기니스,김부선,정광재,최기호,박경진,장재용,전은희,이영필,..."
3,귀신이 산다,2004-09-17,11898748500,11898748500,1875936,1875936,174,27936,한국,(주)시네마서비스,12세관람가,"코미디,공포(호러),판타지",김상진,"김응수,장서희,차승원,김서현,최기웅,유대현,김성미,태인호,임상란,장승현,김정우,김양..."
4,투모로우,2004-06-03,11658584500,11658584500,1830767,1830767,164,22881,미국,이십세기폭스필름코퍼레이션,12세관람가,SF,롤랜드 에머리히,"데니스 퀘이드,제이크 질렌할,이안 홈"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27228,"그대, 고맙소 : 김호중 생애 첫 팬미팅 무비",2020-09-29,-1164993296,-1164993296,102008,102008,106,4771,한국,씨제이포디플렉스 주식회사,전체관람가,공연,없음,김호중
27229,"[통합]회고전, 기획전 등",NaN,985000,501984300,146,104955,1,3,NaN,NaN,NaN,장르없음,없음,NaN
27230,2006 [통합]영화제/기획전/회고전 등,NaN,1938000,583092200,226,106619,1,4,NaN,NaN,NaN,장르없음,없음,NaN
27231,프리즌 이스케이프,2020-05-06,1848297600,1848297600,217056,217056,547,29305,영국,(주)이놀미디어,12세이상관람가,어드벤처,없음,"다니엘 래드클리프,이안 하트"


## 1. 데이터 중복값 제거

+ 영화명과 감독이 같은 데이터에 대해 중복값으로 정의하고 제거

In [28]:
df = df.sort_values(["감독","누적관객수","누적매출액"], ascending=True)
df.drop_duplicates(['영화명',"감독"], keep = 'last',inplace=True)
df

,영화명,개봉일,매출액,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,배급사,등급,장르,감독,배우
26970,"416 프로젝트 ""공동의 기억: 트라우마""",NaN,2192500,2192500,437,437,3,17,한국,NaN,NaN,다큐멘터리,416연대 미디어위원회,NaN
25868,미씨카: 더 네크로맨서,2017-11-02,300000,300000,60,60,1,1,미국,(주)제이에이와이이엔터테인먼트,15세이상관람가,판타지,A. 토드 스미스,"애덤 존슨,로버트 제인,케빈 소르보"
3093,월드 워 핵전쟁의 서막,2019-03-07,26000,26000,17,17,5,17,뉴질랜드,(주)풍경소리,[국내] 15세 관람가,액션,A.K. 스트롬,"모건 브래들리,그레이엄 빈센트,캠벨 루셀"
3094,울프라이징,2016-11-17,300000,300000,60,60,1,1,미국,주식회사 루믹스미디어,15세이상관람가,공포(호러),BC 퍼트니,빌 오버스트 주니어
3095,제3의앵글 : 형형색색대만전,NaN,1830000,1830000,305,305,1,2,대만,NaN,NaN,NaN,BITO,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23597,댄싱 인 자파,NaN,88000,88000,15,15,1,2,미국,NaN,NaN,다큐멘터리,힐라 메달리아,NaN
26526,위기의 30대 여자들,NaN,946400,946400,136,136,1,2,이스라엘,NaN,NaN,다큐멘터리,힐라 메달리아,NaN
23598,16세의 사운드트랙,2020-07-02,3543400,3543400,413,413,19,150,영국,(주)마노엔터테인먼트,12세이상관람가,코미디,힐러리 셰익스피어,"제임스 캘로웨이,스칼렛 마셜"
2786,나의 말을 지켜봐: 어느 귀 먼 연예인들의 이야기,NaN,293500,293500,114,114,1,2,미국,NaN,NaN,다큐멘터리,힐러리 스칼,NaN


## 2. 장르

+ 장르없음을 포함해 25개의 가테고리로 분류
+ 장르가 여러개인 영화에 대해서는 첫번째 장르로 분류

In [29]:
# 장르 데이터 확인

df["장르"].value_counts()

드라마                              4336
멜로/로맨스                           2036
다큐멘터리                            2001
애니메이션                            1810
액션                               1452
장르없음                             1282
코미디                              1025
에로                                943
성인물(에로)                           759
공포(호러)                            549
스릴러                               462
범죄                                366
SF                                244
미스터리                              161
판타지                               132
공포                                132
공연                                129
전쟁                                126
가족                                 93
어드벤처                               82
모험                                 79
뮤지컬                                61
드라마,멜로/로맨스                         57
멜로/로맨스,코미디                         51
코미디,드라마                            47
서부                                 45
공연실황        

In [30]:
# index 초기화

df.reset_index(inplace=True)
df = df.loc[:, "영화명": "배우"]
df

,영화명,개봉일,매출액,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,배급사,등급,장르,감독,배우
0,"416 프로젝트 ""공동의 기억: 트라우마""",NaN,2192500,2192500,437,437,3,17,한국,NaN,NaN,다큐멘터리,416연대 미디어위원회,NaN
1,미씨카: 더 네크로맨서,2017-11-02,300000,300000,60,60,1,1,미국,(주)제이에이와이이엔터테인먼트,15세이상관람가,판타지,A. 토드 스미스,"애덤 존슨,로버트 제인,케빈 소르보"
2,월드 워 핵전쟁의 서막,2019-03-07,26000,26000,17,17,5,17,뉴질랜드,(주)풍경소리,[국내] 15세 관람가,액션,A.K. 스트롬,"모건 브래들리,그레이엄 빈센트,캠벨 루셀"
3,울프라이징,2016-11-17,300000,300000,60,60,1,1,미국,주식회사 루믹스미디어,15세이상관람가,공포(호러),BC 퍼트니,빌 오버스트 주니어
4,제3의앵글 : 형형색색대만전,NaN,1830000,1830000,305,305,1,2,대만,NaN,NaN,NaN,BITO,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21320,댄싱 인 자파,NaN,88000,88000,15,15,1,2,미국,NaN,NaN,다큐멘터리,힐라 메달리아,NaN
21321,위기의 30대 여자들,NaN,946400,946400,136,136,1,2,이스라엘,NaN,NaN,다큐멘터리,힐라 메달리아,NaN
21322,16세의 사운드트랙,2020-07-02,3543400,3543400,413,413,19,150,영국,(주)마노엔터테인먼트,12세이상관람가,코미디,힐러리 셰익스피어,"제임스 캘로웨이,스칼렛 마셜"
21323,나의 말을 지켜봐: 어느 귀 먼 연예인들의 이야기,NaN,293500,293500,114,114,1,2,미국,NaN,NaN,다큐멘터리,힐러리 스칼,NaN


In [31]:
# 장르가 결측인 데이터는 장르없음, 장르가 여러개인 데이터는 가장 앞 장르만 남기기

for i in range(len(df["장르"])):
    if type(df["장르"][i]) == float:
        df["장르"][i] = "장르없음"
    else:
        df["장르"][i] = df["장르"][i].split(",")[0]

<ipython-input-31-a95e4175e87a>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["장르"][i] = df["장르"][i].split(",")[0]
<ipython-input-31-a95e4175e87a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["장르"][i] = "장르없음"


In [32]:
# 비슷한 장르끼리 묶기

df.replace("성인물(에로)", "에로", inplace = True)
df.replace("서부극(웨스턴)", "서부", inplace = True)
df.replace("공연실황", "공연", inplace = True)
df.replace("서스펜스", "스릴러", inplace = True)
df.replace("공포(호러)", "공포", inplace = True)
df.replace("멜로/로맨스", "멜로로맨스", inplace = True)
df.replace("실험", "다큐멘터리", inplace = True)

In [33]:
# 기타장르 제거

etc_index = df[df["장르"]=="기타"].index
df.drop(etc_index, inplace=True)

In [34]:
# 총 25개의 장르 확인

df["장르"].value_counts()

드라마      4552
장르없음     3125
멜로로맨스    2167
다큐멘터리    2014
애니메이션    1899
에로       1711
액션       1586
코미디      1173
공포        756
스릴러       510
범죄        410
SF        295
미스터리      180
공연        172
판타지       149
전쟁        132
가족        117
어드벤처      102
모험         79
뮤지컬        68
서부         56
사극         53
느와르         9
무협          8
서사          2
Name: 장르, dtype: int64

## 3. 등급

+ 등급없음 포함해 5개의 카테고리로 분류
+ 청소년관람불가, 15세이상관람가, 12세이상관람가, 전체관람가, 등급없음

In [35]:
# 등급 데이터 확인

df['등급'].value_counts()

청소년관람불가                             3884
15세이상관람가                            2647
[국내] 청소년 관람불가                       1976
12세이상관람가                            1484
전체관람가                               1435
[국내] 15세 관람가                         530
15세관람가                               412
[국내] 전체 관람가                          367
[국내] 12세 관람가                         281
12세관람가                               247
18세관람가                               220
15세 관람가                              191
청소년 관람불가                             156
12세 관람가                              106
전체 관람가                                55
연소자관람불가                               35
고등학생이상관람가                             34
연소자관람가                                19
 15세 관람가                              18
 청소년 관람불가                             18
중학생이상관람가                              15
고등학생이상관람가,15세이상관람가                    13
15세관람가,15세이상관람가                       12
연소자관람불가,청소년관람불가                       10
연소자관람가,전체관람가    

In [36]:
# 등급이 결측인 데이터는 등급없음으로 처리

df['등급'] = df['등급'].fillna("등급없음")

In [37]:
# 등급이 이상한 데이터 처리

df.replace([" 청소년 관람불가 ","청소년 관람불가 ","청소년 관람불가","[국내] 청소년 관람불가","고등학생이상관람가,청소년관람불가","청소년관람불가,12세관람가","국민학생관람불가,청소년관람불가","청소년관람불가,고등학생이상관람가","18세관람가,청소년관람불가","18세 미만인 자는 관람할 수 없는 등급","미성년자관람불가",'18세관람가', "연소자관람불가","연소자관람불가,청소년관람불가"], '청소년관람불가',inplace=True)
df.replace([" 15세 관람가 ","15세 관람가 ","[국내] 15세 관람가"," 15세 관람가","15세관람가","고등학생이상관람가","고등학생이상관람가,15세이상관람가","15세 미만인 자는 관람할 수 없는 등급","15세관람가,15세이상관람가","15세이상관람가,중학생이상관람가","15세이상관람가,미성년자관람불가","청소년관람불가,15세이상관람가","연소자관람불가,15세이상관람가","국민학생관람불가,15세이상관람가","15세이상관람가,전체관람가","15세이상관람가,18세 미만인 자는 관람할 수 없는 등급","연소자관람가,15세이상관람가","18세관람가,15세이상관람가","15세 미만인 자는 관람할 수 없는 등급 ,15세이상관람가"], '15세이상관람가',inplace=True)
df.replace([" 12세 관람가 ","12세 관람가 "," 12세 관람가","[국내] 12세 관람가","12세관람가","중학생이상관람가","국민학생관람불가","12세이상관람가,12세관람가","12세 미만인 자는 관람할 수 없는 등급","12세이상관람가,중학생이상관람가","12세이상관람가,국민학생관람불가","12세이상관람가,전체관람가","12세이상관람가,15세 미만인 자는 관람할 수 없는 등급","12세이상관람가,15세이상관람가","12세이상관람가,고등학생이상관람가","12세이상관람가,국민학생관람불가,15세이상관람가","12세이상관람가,연소자관람가","15세관람가,12세이상관람가"],"12세이상관람가", inplace=True)
df.replace(["전체관람가,중학생이상관람가"," 전체 관람가 ","전체 관람가 ","[국내] 전체 관람가","연소자관람가","모든 관람객이 관람할 수 있는 등급","연소자관람가,전체관람가","청소년관람불가,전체관람가","미성년자관람가","모든 관람객이 관람할 수 있는 등급,전체관람가","12세이상관람가,연소자관람가,전체관람가"],"전체관람가",inplace=True)

In [38]:
# 총 5개의 등급 확인

df["등급"].value_counts()

등급없음        7059
청소년관람불가     6316
15세이상관람가    3887
12세이상관람가    2173
전체관람가       1890
Name: 등급, dtype: int64

## 4. 대표국적

+ 개봉한 영화수에 따라 상위 10개 국적과 기타국적으로 분류
+ 미국, 한국, 일본, 기타, 프랑스, 영국, 독일, 중국, 이탈리아, 캐나다, 스페인

In [39]:
# 상위 10개 국가가 아닌 데이터에 대해 기타로 처리

nation = ["미국", "한국", "일본", "프랑스", "영국", "독일", "중국", "이탈리아", "캐나다", "스페인"]

for i in range(len(df)):
    if df.iloc[i]["대표국적"] not in nation:
        df["대표국적"].replace(df.iloc[i]["대표국적"],"기타",inplace=True)

In [40]:
# 총 11개의 대표국적 확인

df["대표국적"].value_counts()

미국      4997
한국      4938
일본      4194
기타      3396
프랑스     1119
영국       839
독일       452
중국       437
이탈리아     390
캐나다      305
스페인      258
Name: 대표국적, dtype: int64

## 5. 배급사

+ 결측인 데이터는 0으로 처리
+ 배급사가 여러개인 데이터는 1개만 사용

In [92]:
# 결측인 데이터는 0으로, 배급사가 여러개인 데이터는 가장 앞 배급사만 남기기

for i in range(len(df["배급사"])):
    if type(df["배급사"][i]) == float:
        df["배급사"][i] = 0
    else:
        df["배급사"][i] = str(df["배급사"][i]).split(",")[0]

<ipython-input-92-0a4346291d83>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["배급사"][i] = str(df["배급사"][i]).split(",")[0]


In [93]:
# 배급사 공백 제거

for i in range(len(df["배급사"])):
    if type(df["배급사"][i]) == float:
        df["배급사"][i] = 0
    else:
        List = df["배급사"][i].split(" ")
        df["배급사"][i] = "".join(List)

<ipython-input-93-f92b1096c836>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["배급사"][i] = "".join(List)


In [94]:
# 같은 회사, 같은 계열사 처리

df["배급사"].replace(["씨제이씨지브이(CJCGV)(주)", "CJ엔터테인먼트", "CJE&M(주)투니버스", "CJE&MPictures", "씨제이포디플렉스주식회사", "(주)씨제이이엔엠"], "CJENM", inplace = True)
df["배급사"].replace(["(주)시네마서비스", "시네마서비스", "필라멘트픽쳐스", "투니버스(주)온미디어", "CGV아트하우스", "CGVICECON"], "CJENM", inplace = True)
df["배급사"].replace(["롯데컬처웍스(주)롯데엔터테인먼트", "롯데쇼핑㈜롯데시네마", "롯데엔터테인먼트"], "롯데쇼핑㈜롯데엔터테인먼트", inplace = True)
df["배급사"].replace(["소니픽쳐스릴리징월트디즈니스튜디오스코리아(주)", "소니픽쳐스릴리징코리아", "소니픽쳐스엔터테인먼트코리아주식회사극장배급지점"], "한국소니픽쳐스릴리징브에나비스타영화㈜", inplace = True)
df["배급사"].replace(["㈜메가박스", "메가박스", "메가박스중앙(주)", "메가박스중앙(주)MEGABOX"], "메가박스중앙(주)플러스엠", inplace = True)
df["배급사"].replace(["가을엔터테인먼트"], "(주)영화사가을", inplace = True)
df["배급사"].replace(["(주)예지림엔터테인먼트"], "예지림엔터테인먼트", inplace = True)
df["배급사"].replace(["디스테이션"], "(주)디스테이션 ", inplace = True)
df["배급사"].replace(["(주)씨지브이신도림"], "(주)프라임엔터테인먼트", inplace = True)
df["배급사"].replace(["NEW","(주)NEW","(주)콘텐츠판다", "콘텐츠판다"],"(주)넥스트엔터테인먼트월드(NEW)",inplace=True)
df["배급사"].replace(["이십세기폭스필름코퍼레이션", "20세기폭스코리아"],"이십세기폭스코아(주)",inplace=True)
df["배급사"].replace(["시네마서비스", "플레너스(주)시네마서비스"],"(주)시네마서비스",inplace=True)
df["배급사"].replace(["월트디즈니컴퍼니코리아(주)", "월트디즈니코리아㈜", "월트디즈니컴퍼니코리아"],"월트디즈니컴퍼니코리아유한책임회사",inplace=True)
df["배급사"].replace(["(주)코리아스크린"],"㈜코리아스크린",inplace=True)
df["배급사"].replace(["(주)풍경소리"],"주식회사풍경소리",inplace=True)
df["배급사"].replace(["시네마리퍼블릭"],"(주)시네마리퍼블릭",inplace=True)
df["배급사"].replace(["레드아이스"],"주식회사레드아이스",inplace=True)

df["배급사"].replace("유니버설픽쳐스인터내셔널코리아",inplace=True)
df["배급사"].replace(["(주)마운틴픽쳐스","마운틴픽처스","마운틴픽쳐스"],"주식회사마운틴픽쳐스",inplace=True)
df["배급사"].replace("팝엔터테인먼트","(주)팝엔터테인먼트",inplace=True)
df["배급사"].replace("디스테이션","(주)디스테이션 ",inplace=True)
df["배급사"].replace("드림팩트엔터테인먼트","(주)드림팩트엔터테인먼트",inplace=True)
df["배급사"].replace("나우콘텐츠","(주)나우콘텐츠",inplace=True)
df["배급사"].replace(["㈜싸이더스픽쳐스","싸이더스"],"(주)싸이더스",inplace=True)
df["배급사"].replace("스폰지","(주)스폰지이엔티",inplace=True)
df["배급사"].replace(["이수C&E","이수문화컨텐츠투자조합"],"(주)이수C&E",inplace=True)
df["배급사"].replace("얼리버드","(주)얼리버드픽쳐스",inplace=True)
df["배급사"].replace("(주)안다미로","(주)영화사안다미로",inplace=True)
df["배급사"].replace("(주)루믹스미디어","주식회사루믹스미디어",inplace=True)
df["배급사"].replace(["워너브러더스사㈜","워너브러더스픽쳐스"],"워너브러더스코리아(주)",inplace=True)
df["배급사"].replace("인디스토리","㈜인디스토리",inplace=True)
df["배급사"].replace("인디플러그","(주)인디플러그",inplace=True)
df["배급사"].replace(["대원디지털엔터테인먼트","대원씨앤에이홀딩스"],"대원미디어(주)",inplace=True)
df["배급사"].replace("소나무미디어","(주)소나무픽쳐스",inplace=True)
df["배급사"].replace("씨네그루(주)키다리이엔티","(주)키다리이엔티",inplace=True)
df["배급사"].replace("(주)브릿지웍스","브릿지웍스엔터테인먼트(주)",inplace=True)

df["배급사"].replace('(주)루믹스미디어', '주식회사루믹스미디어', inplace=True)
df["배급사"].replace('(주)안다미로', '(주)영화사안다미로', inplace=True)
df["배급사"].replace('드림팩트엔터테인먼트', '(유)조이앤컨텐츠그룹', inplace=True)
df["배급사"].replace('(주)드림팩트엔터테인먼트', '(유)조이앤컨텐츠그룹', inplace=True)
df["배급사"].replace('디스테이션', '(주)디스테이션', inplace=True)
df["배급사"].replace('플레너스(주)시네마서비스', '(주)시네마서비스', inplace=True)
df["배급사"].replace('시네마서비스', '(주)시네마서비스', inplace=True)
df["배급사"].replace('조이앤시네마', '(유)조이앤컨텐츠그룹', inplace=True)
df["배급사"].replace('(유)조이앤시네마', '(유)조이앤컨텐츠그룹', inplace=True)
df["배급사"].replace('(주)케이알씨지', '(유)조이앤컨텐츠그룹', inplace=True)

In [95]:
# 무슨 코드인지 기억이 안남

count = df["배급사"].value_counts()

movies = []

for key, value in count.items():
    if value == 1:
        movies.append(key)
        
for i in range(len(df)):
    if df.iloc[i]["배급사"] in movies:
        df["배급사"].replace(df.iloc[i]["배급사"], 0, inplace=True)

In [97]:
# 문자열 0으로 처리된 값을 숫자형으로 바꾸기

df['배급사'].replace("0", 0, inplace = True)

In [98]:
# 배급화 확인

df['배급사'].value_counts()

0                          10888
(주)도키엔터테인먼트                  669
CJENM                        618
(유)조이앤컨텐츠그룹                  555
(주)영진크리에이티브                  403
롯데쇼핑㈜롯데엔터테인먼트                263
(주)소나무픽쳐스                    228
한국소니픽쳐스릴리징브에나비스타영화㈜          219
워너브러더스코리아(주)                 178
유니버설픽쳐스인터내셔널코리아(유)           177
(주)팝엔터테인먼트                   173
(주)쇼박스                       165
(주)영화사진진                     153
(주)넥스트엔터테인먼트월드(NEW)          144
이십세기폭스코리아(주)                 130
(주)영화사가을                     126
스크린조이                        122
(주)플라이어                      122
주식회사마운틴픽쳐스                   119
(주)스폰지이엔티                    112
와이드릴리즈(주)                    103
판씨네마(주)                       87
(주)디스테이션                      84
(주)미디어데이                      79
(주)박수엔터테인먼트                   76
주식회사루믹스미디어                    75
㈜인디스토리                        75
(주)영화사안다미로                    74
(주)엣나인필름                      73
(주)나우콘텐츠                      72
(주)영화사빅   

## 6. 감독

+ 결측인 데이터는 0으로 처리
+ 감독 데이터가 여러개인 경우에는 1개만 사용

In [46]:
# 결측인 데이터는 0으로, 감독이 여러개인 데이터는 가장 앞 감독만 남기기

for i in range(len(df["감독"])):
    if type(df["감독"][i]) == float:
        df["감독"][i] = 0
    else:
        df["감독"][i] = df["감독"][i].split(",")[0]

<ipython-input-46-af0522f50a86>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["감독"][i] = df["감독"][i].split(",")[0]


In [47]:
# 감독 데이터 공백 제거

for i in range(len(df["감독"])):
    if type(df["감독"][i]) == float:
        df["감독"][i] = 0
    else:
        List = df["감독"][i].split(" ")
        df["감독"][i] = "".join(List)

<ipython-input-47-b29abfc2ba11>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["감독"][i] = "".join(List)


In [53]:
# 감독데이터에 없음과 개봉연도없음을 숫자 0으로 바꾸기

df["감독"].replace(["없음", "개봉연도없음"], 0, inplace = True)

In [60]:
# 무슨 코드인지 기억이 안남

count = df["감독"].value_counts()

movies = []

for key, value in count.items():
    if value == 1:
        movies.append(key)
        
for i in range(len(df)):
    if df.iloc[i]["감독"] in movies:
        df["감독"].replace(df.iloc[i]["감독"], 0, inplace=True)

## 7. 배우

+ 결측인 데이터는 0으로 처리
+ 배우 데이터가 여러개인 경우에는 3개만 사용

In [87]:
# 배우 데이터 공백 제거

for i in range(len(df["배우"])):
    if type(df["배우"][i]) == float:
        df["배우"][i] = 0
    else:
        List = str(df["배우"][i]).split(" ")
        df["배우"][i] = "".join(List)

<ipython-input-87-d1b80ff8daa7>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["배우"][i] = "".join(List)


In [88]:
# 배우 3명만 남기기

for i in range(len(df["배우"])):
    if type(df["배우"][i]) == float:
        df["배우"][i] = 0
    else:
        df["배우"][i] = df["배우"][i].split(",")[0:3]

<ipython-input-88-a20a9924b785>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["배우"][i] = df["배우"][i].split(",")[0:3]


## 8. 매출액 / 누적매출액 / 관객수 / 누적관객수

+ 음수인 값을 양수로 만들기

In [63]:
values = []

for value in df['매출액'].values:
    value = abs(value)
    values.append(value)

In [64]:
stacked_values = []

for value in df['누적매출액'].values:
    value = abs(value)
    stacked_values.append(value)

In [65]:
df['매출액'] = values
df['누적매출액'] = stacked_values

In [100]:
values = []

for value in df['관객수'].values:
    value = abs(value)
    values.append(value)

In [101]:
stacked_values = []

for value in df['누적관객수'].values:
    value = abs(value)
    stacked_values.append(value)

In [102]:
df['관객수'] = values
df['누적관객수'] = stacked_values

## 9. 개봉일

+ 개봉일이 없는 데이터는 1900-01-01로 만들기
+ 월과 일이 누락된 데이터는 -01 붙이기

In [67]:
pd.options.display.float_format = '{:.5f}'.format

In [68]:
# 결측인 데이터에 1900-01-01로 만들기

df['개봉일'].fillna("1900-01-01", inplace = True)

In [70]:
# 개봉일 공백 제거

for i in range(len(df["개봉일"])):
    if type(df["개봉일"][i]) == float:
        df["개봉일"][i] = 0
    else:
        List = df["개봉일"][i].split(" ")
        df["개봉일"][i] = "".join(List)

<ipython-input-70-00e93f1db0c1>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["개봉일"][i] = "".join(List)


In [73]:
# 날짜 형식이 0000-00-00이 아닌 행의 인덱스 가져오기

index=[]

for i in range(len(df["개봉일"])):
    p= re.compile("\d\d\d\d-\d\d-\d\d")
    m= bool(p.match(str(df["개봉일"][i])))
    if m == False:
            index.append(i)

In [74]:
# 날짜형식이 yyyy-mm-dd아닌 데이터 수정

for j in range(len(df['개봉일'])):
    if len(df.iloc[j]['개봉일']) == 4:
        df['개봉일'][j] = df.iloc[j]['개봉일'] + "-01-01"

    elif len(df.iloc[j]['개봉일']) == 7:
        df['개봉일'][j] = df.iloc[j]['개봉일'] + "-01"
        
        
    elif  len(df.iloc[j]['개봉일']) >= 11:
        df['개봉일'][j] = df.iloc[j]['개봉일'][-10:]
        
    else: 
        continue

<ipython-input-74-c82bacef071e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['개봉일'][j] = df.iloc[j]['개봉일'] + "-01"
<ipython-input-74-c82bacef071e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['개봉일'][j] = df.iloc[j]['개봉일'] + "-01-01"
<ipython-input-74-c82bacef071e>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['개봉일'][j] = df.iloc[j]['개봉일'][-10:]


In [90]:
df.head(100)

,영화명,개봉일,매출액,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,배급사,등급,장르,감독,배우
0,"416 프로젝트 ""공동의 기억: 트라우마""",1900-01-01,2192500,2192500,437,437,3,17,한국,0,등급없음,다큐멘터리,0,[0]
1,미씨카: 더 네크로맨서,2017-11-02,300000,300000,60,60,1,1,미국,(주)제이에이와이이엔터테인먼트,15세이상관람가,판타지,0,"[애덤존슨, 로버트제인, 케빈소르보]"
2,월드 워 핵전쟁의 서막,2019-03-07,26000,26000,17,17,5,17,기타,(주)풍경소리,15세이상관람가,액션,0,"[모건브래들리, 그레이엄빈센트, 캠벨루셀]"
3,울프라이징,2016-11-17,300000,300000,60,60,1,1,미국,주식회사 루믹스미디어,15세이상관람가,공포,0,[빌오버스트주니어]
4,제3의앵글 : 형형색색대만전,1900-01-01,1830000,1830000,305,305,1,2,기타,0,등급없음,장르없음,0,[0]
5,연쇄살인마의 이중생활,2020-06-18,16000,16000,7,7,5,7,스페인,주식회사 마운틴픽쳐스,15세이상관람가,범죄,0,"[안나알렌, 나초프레스네다, 메르세데스카스트로]"
6,몬스터 아일랜드,2017-10-27,3000,3000,1,1,1,1,미국,주식회사 레드아이스,15세이상관람가,어드벤처,0,[존W.바우어스]
7,잃어버린 세계를 찾아 2015,2015-12-03,4000,4000,1,1,1,1,미국,(주)케이알씨지,12세이상관람가,어드벤처,0,"[C.토마스하우웰, 티모시바톰즈]"
8,과부: 탐스러운 육체,2016-11-19,5000,5000,1,1,1,1,일본,(주)영진크리에이티브,청소년관람불가,멜로로맨스,0,"[타치바나케이코, 이치죠리노]"
9,페이스트리의 왕,1900-01-01,642000,642000,107,107,1,2,미국,0,등급없음,장르없음,0,[0]


In [103]:
df.to_csv("테스트파일.csv", encoding = "utf-8-sig", index = False)